# Odtwarzalność w module TensorFlow

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

/Users/ageron/.virtualenvs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Lista kontrolna

1. Nie uruchamiaj modułu TensorFlow na procesorze GPU.
2. Uważaj na wielowątkowość i przetwarzaj moduł TensorFlow wyłącznie w jednym wątku.
3. Wyznacz wszystkie ziarna losowości.
4. Wyeliminuj pozostałe źródła zmienności. 

## Nie uruchamiaj modułu TensorFlow na procesorze GPU.

Niektóre operacje (takie jak `tf.reduce_sum()`) faworyzują wydajność ponad precyzją, a ich wyniki mogą nieco różnić się w poszczególnych przebiegach. Aby uzyskiwać odtwarzalne rezultaty spraw, żeby moduł TensorFlow działał na jednostce CPU:

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

## Uważaj na wielowątkowość

Liczby zmiennoprzecinkowe mają ograniczoną precyzję, dlatego kolejność wykonywania działań ma znaczenie:

In [3]:
2. * 5. / 7.

1.4285714285714286

In [4]:
2. / 7. * 5.

1.4285714285714284

Spraw, żeby moduł TensorFlow realizował operacje w jednym wątku:

In [5]:
config = tf.ConfigProto(intra_op_parallelism_threads=1,
                        inter_op_parallelism_threads=1)

with tf.Session(config=config) as sess:
    #... dalsza część kodu będzie realizowana w jednym wątku
    pass

Pule wątków dla wszystkich sesji są tworzone podczas tworzenia pierwszej sesji, zatem wszystkie pozostałe sesje w tym notatniku będą jednowątkowe:

In [6]:
with tf.Session() as sess:
    #... także jednowątkowo!
    pass

## Wyznacz wszystkie ziarna losowości!

### Funkcja `hash()` Pythona

In [7]:
print(set("Zrestartuj jądro i uruchom ten listing ponownie"))
print(set("Zrestartuj jądro i uruchom ten listing ponownie"))

{'n', 'k', 'l', 'h', 'r', 'a', 'i', 't', 'd', 's', 'g', 'T', ' ', 'y', 'e', 'u'}
{'n', 'k', 'l', 'h', 'r', 'a', 'i', 't', 'd', 's', 'g', 'T', ' ', 'y', 'e', 'u'}


Od wersji 3.3 języka Python wynik będzie za każdym razem inny, chyba że uruchomisz środowisko Python z wartością `0` zmiennej środowiskowej `PYTHONHASHSEED`:

```shell
PYTHONHASHSEED=0 python
```

```pycon
>>> print(set("Teraz wynik jest stabilny w poszczególnych przebiegach"))
{'n', 'b', 'h', 'o', 'i', 'a', 'r', 't', 'p', 'N', 's', 'c', ' ', 'l', 'e', 'w', 'u'}
>>> exit()
```

```shell
PYTHONHASHSEED=0 python
```
```pycon
>>> print(set("Teraz wynik jest stabilny w poszczególnych przebiegach"))
{'n', 'b', 'h', 'o', 'i', 'a', 'r', 't', 'p', 'N', 's', 'c', ' ', 'l', 'e', 'w', 'u'}
```

Ewentualnie możesz wyznaczyć tę zmienną środowiskową w sposób systemowy, ale prawdopodobnie nie jest to dobry pomysł, ponieważ ta automatyczna losowość została [wprowadzona z przyczyn bezpieczeństwa](http://ocert.org/advisories/ocert-2011-003.html).

Niestety, wyznaczenie tej zmiennej środowiskowej w Pythonie (np. za pomocą `os.environ["PYTHONHASHSEED"]="0"`) nie będzie działać, ponieważ Python odczytuje ją w momencie rozruchu. W przypadku notatników Jupyter należy uruchomić serwer Jupyter w następujący sposób:

```shell
PYTHONHASHSEED=0 jupyter notebook
```

In [8]:
if os.environ.get("PYTHONHASHSEED") != "0":
    raise Exception("Musisz wyznaczyć PYTHONHASHSEED=0 w momencie uruchomienia serwera Jupyter, aby uzyskiwać odtwarzalne wyniki.")

### Generatory liczb losowych (ang. Random Number Generators - RNGs)

In [9]:
import random

random.seed(42)
print(random.random())
print(random.random())

print()

random.seed(42)
print(random.random())
print(random.random())

0.6394267984578837
0.025010755222666936

0.6394267984578837
0.025010755222666936


### Generatory liczb losowych w module NumPy

In [10]:
import numpy as np

np.random.seed(42)
print(np.random.rand())
print(np.random.rand())

print()

np.random.seed(42)
print(np.random.rand())
print(np.random.rand())

0.3745401188473625
0.9507143064099162

0.3745401188473625
0.9507143064099162


### Generatory liczb losowych w module TensorFlow

Działanie modułu TensorFlow jest bardziej skomplikowane z dwóch powodów:
* Tworzysz graf, a później go realizujesz. Należy wyznaczyć ziarno losowości przed utworzeniem losowych operacji.
* Występują tu dwa ziarna losowości: jedno na poziomie grafu, a drugie na poziomie poszczególnych operacji.

In [11]:
import tensorflow as tf

tf.set_random_seed(42)
rnd = tf.random_uniform(shape=[])

with tf.Session() as sess:
    print(rnd.eval())
    print(rnd.eval())

print()

with tf.Session() as sess:
    print(rnd.eval())
    print(rnd.eval())

0.63789964
0.8774011

0.63789964
0.8774011


Za każdym razem, gdy resetujesz graf, musisz ponownie wyznaczyć źródło:

In [12]:
tf.reset_default_graph()

tf.set_random_seed(42)
rnd = tf.random_uniform(shape=[])

with tf.Session() as sess:
    print(rnd.eval())
    print(rnd.eval())

print()

with tf.Session() as sess:
    print(rnd.eval())
    print(rnd.eval())

0.63789964
0.8774011

0.63789964
0.8774011


Jeżeli tworzysz własny graf, będzie on ignorował ziarno domyślnego grafu:

In [13]:
tf.reset_default_graph()
tf.set_random_seed(42)

graph = tf.Graph()
with graph.as_default():
    rnd = tf.random_uniform(shape=[])

with tf.Session(graph=graph):
    print(rnd.eval())
    print(rnd.eval())

print()

with tf.Session(graph=graph):
    print(rnd.eval())
    print(rnd.eval())

0.5718187
0.6233171

0.32140207
0.46593904


Musisz wyznaczyć własne ziarno:

In [14]:
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(42)
    rnd = tf.random_uniform(shape=[])

with tf.Session(graph=graph):
    print(rnd.eval())
    print(rnd.eval())

print()

with tf.Session(graph=graph):
    print(rnd.eval())
    print(rnd.eval())

0.63789964
0.8774011

0.63789964
0.8774011


Jeżeli wyznaczysz ziarno po utworzeniu losowej operacji, nie będzie miało ono na nią wpływu:

In [15]:
tf.reset_default_graph()

rnd = tf.random_uniform(shape=[])

tf.set_random_seed(42) # ŹLE, BEZ EFEKTU!
with tf.Session() as sess:
    print(rnd.eval())
    print(rnd.eval())

print()

tf.set_random_seed(42) # ŹLE, BEZ EFEKTU!
with tf.Session() as sess:
    print(rnd.eval())
    print(rnd.eval())

0.087068915
0.6322479

0.17158246
0.2868148


#### Uwaga na temat ziaren używanych w operacjach

Możesz także wyznaczyć ziarno dla poszczególnych operacji losowych. Będzie ono łączone z ziarnem grafowym, co pozwoli uzyskać ziarno końcowe, używane przez tę operację. Mechanizm ten został podsumowany w poniższej tabeli:

| Ziarno grafu | Ziarno operacji |  Uzyskane ziarno               |
|--------------|-----------------|--------------------------------|
| Brak         | Brak            | Losowe                         |
| graph_seed   | Brak            | f(graph_seed, op_index)        |
| Brak         | op_seed         | f(default_graph_seed, op_seed) |
| graph_seed   | op_seed         | f(graph_seed, op_seed)         |

* `f()` jest funkcją deterministyczną.
* `op_index = graph._last_id`. Gdy istnieje ziarno graf, różne operacje losowe pozbawione ziaren będą dawały odmienne wyniki, jednak każda z nich będzie miała w każdym przebiegu taką samą sekwencję wyników.

W trybie ochoczym miejsce ziarna grafu zajmuje ziarno globalne (gdyż w trybie tym nie istnieje graf). 

In [16]:
tf.reset_default_graph()

rnd1 = tf.random_uniform(shape=[], seed=42)
rnd2 = tf.random_uniform(shape=[], seed=42)
rnd3 = tf.random_uniform(shape=[])

with tf.Session() as sess:
    print(rnd1.eval())
    print(rnd2.eval())
    print(rnd3.eval())
    print(rnd1.eval())
    print(rnd2.eval())
    print(rnd3.eval())

print()

with tf.Session() as sess:
    print(rnd1.eval())
    print(rnd2.eval())
    print(rnd3.eval())
    print(rnd1.eval())
    print(rnd2.eval())
    print(rnd3.eval())

0.95227146
0.95227146
0.55099714
0.8960779
0.8960779
0.54318357

0.95227146
0.95227146
0.6398845
0.8960779
0.8960779
0.24617589


W poniższym przykładzie może się wydawać, że wszystkie operacje losowe mają wyznaczone to samo ziarno losowości, ale w rzeczywistości `rnd3` zawiera inne ziarno:

In [17]:
tf.reset_default_graph()

tf.set_random_seed(42)

rnd1 = tf.random_uniform(shape=[], seed=42)
rnd2 = tf.random_uniform(shape=[], seed=42)
rnd3 = tf.random_uniform(shape=[])

with tf.Session() as sess:
    print(rnd1.eval())
    print(rnd2.eval())
    print(rnd3.eval())
    print(rnd1.eval())
    print(rnd2.eval())
    print(rnd3.eval())

print()

with tf.Session() as sess:
    print(rnd1.eval())
    print(rnd2.eval())
    print(rnd3.eval())
    print(rnd1.eval())
    print(rnd2.eval())
    print(rnd3.eval())

0.4163028
0.4163028
0.96100175
0.033224702
0.033224702
0.17637014

0.4163028
0.4163028
0.96100175
0.033224702
0.033224702
0.17637014


#### Interfejs estymatorów

**Wskazówka**: w notatniku Jupyter prawdopodobnie będziesz chciał wyznaczać ziarno losowości regularnie, dzięki czemu będziesz mógł wracać i uruchamiać notatnik od tego miejscu (a nie od początku) i nadal uzyskiwać odtwarzalne wyniki.

In [18]:
random.seed(42)
np.random.seed(42)
tf.set_random_seed(42)

Jeżeli korzystasz z interfejsu estymatorów (Estimators API), stwórz `RunConfig` i wyznacz jego `tf_random_seed`, a następnie przekaż go do konstruktora estymatora:

In [19]:
my_config = tf.estimator.RunConfig(tf_random_seed=42)

feature_cols = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
dnn_clf = tf.estimator.DNNClassifier(hidden_units=[300, 100], n_classes=10,
                                     feature_columns=feature_cols,
                                     config=my_config)

INFO:tensorflow:Using config: {'_model_dir': '/var/folders/wy/h39t6kb11pnbb0pzhksd_fqh0000gn/T/tmp2xxrubio', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11dba7da0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Sprawdźmy to na zestawie danych MNIST:

In [20]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)

Niestety, `numpy_input_fn` nie pozwala na wyznaczanie ziarna, gdy `shuffle=True`, dlatego musimy samodzielnie przetasować dane i wyznaczyć `shuffle=False`.

In [21]:
indices = np.random.permutation(len(X_train))
X_train_shuffled = X_train[indices]
y_train_shuffled = y_train[indices]

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train_shuffled}, y=y_train_shuffled, num_epochs=10, batch_size=32, shuffle=False)
dnn_clf.train(input_fn=input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/wy/h39t6kb11pnbb0pzhksd_fqh0000gn/T/tmp2xxrubio/model.ckpt.
INFO:tensorflow:loss = 73.945915, step = 1
INFO:tensorflow:global_step/sec: 348.999
INFO:tensorflow:loss = 21.020527, step = 101 (0.287 sec)
INFO:tensorflow:global_step/sec: 431.365
INFO:tensorflow:loss = 8.926933, step = 201 (0.232 sec)
INFO:tensorflow:global_step/sec: 438.11
INFO:tensorflow:loss = 2.3184745, step = 301 (0.228 sec)
INFO:tensorflow:global_step/sec: 437.696
INFO:tensorflow:loss = 10.654381, step = 401 (0.228 sec)
INFO:tensorflow:global_step/sec: 452.808
INFO:tensorflow:loss = 4.2829914, step = 501 (0.221 sec)
INFO:tensorflow:global_step/sec: 450.062
INFO:tensorflow:loss = 2.497019, step = 601 (0.222 sec)
INFO:tensorflow:

Końcowa funkcja straty powinna wynosić dokładnie 0,46282205.

Zamiast korzystać z funkcji `numpy_input_fn()` (która nie odtwarza tasowania zestawu danych w każdej epoce), możesz stworzyć własną funkcję wejściową za pomoca interfejsu danych i wyznaczyć ziarno tasowania:

In [22]:
def create_dataset(X, y=None, n_epochs=1, batch_size=32,
                   buffer_size=1000, seed=None):
    dataset = tf.data.Dataset.from_tensor_slices(({"X": X}, y))
    dataset = dataset.repeat(n_epochs)
    dataset = dataset.shuffle(buffer_size, seed=seed)
    return dataset.batch(batch_size)

input_fn=lambda: create_dataset(X_train, y_train, seed=42)

In [23]:
random.seed(42)
np.random.seed(42)
tf.set_random_seed(42)

my_config = tf.estimator.RunConfig(tf_random_seed=42)

feature_cols = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
dnn_clf = tf.estimator.DNNClassifier(hidden_units=[300, 100], n_classes=10,
                                     feature_columns=feature_cols,
                                     config=my_config)
dnn_clf.train(input_fn=input_fn)

INFO:tensorflow:Using config: {'_model_dir': '/var/folders/wy/h39t6kb11pnbb0pzhksd_fqh0000gn/T/tmpawwl1lf0', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x137e1c6d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/wy/h39t6kb11pnbb0pzhksd

Końcowa funkcja straty powinna wynosić dokładnie 1,0556093.

```python
indices = np.random.permutation(len(X_train))
X_train_shuffled = X_train[indices]
y_train_shuffled = y_train[indices]

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train_shuffled}, y=y_train_shuffled,
    num_epochs=10, batch_size=32, shuffle=False)
dnn_clf.train(input_fn=input_fn)
```

#### Interfejs Keras

Jeżeli korzystasz z interfejsu Keras, wystarczy wyznaczać ziarno losowości po każdym wyczyszczeniu sesji:

In [24]:
keras.backend.clear_session()

random.seed(42)
np.random.seed(42)
tf.set_random_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 5s 78us/step - loss: 0.5929 - acc: 0.8450
Epoch 2/10
60000/60000 [==============================] - 4s 75us/step - loss: 0.2804 - acc: 0.9199
Epoch 3/10
60000/60000 [==============================] - 4s 74us/step - loss: 0.2276 - acc: 0.9350
Epoch 4/10
60000/60000 [==============================] - 4s 74us/step - loss: 0.1933 - acc: 0.9449
Epoch 5/10
60000/60000 [==============================] - 4s 74us/step - loss: 0.1682 - acc: 0.9518
Epoch 6/10
60000/60000 [==============================] - 4s 74us/step - loss: 0.1490 - acc: 0.9573
Epoch 7/10
60000/60000 [==============================] - 4s 74us/step - loss: 0.1332 - acc: 0.9622
Epoch 8/10
60000/60000 [==============================] - 5s 75us/step - loss: 0.1202 - acc: 0.9658
Epoch 9/10
60000/60000 [==============================] - 4s 75us/step - loss: 0.1090 - acc: 0.9693
Epoch 10/10
60000/60000 [==============================] - 4s 75us/step - loss: 0.1000 - acc: 0.9716

Na koniec uczenia powinieneś uzyskać dokładnie 97,16% dokładności dla zbioru uczącego.

## Wyeliminuj pozostałe źródła zmienności

Na przykład funkcja `os.listdir()` zwraca nazwy plików w kolejności uzależnionej od sposobu ich indeksowania przez system plików:

In [25]:
for i in range(10):
    with open("moj_test_foo_{}".format(i), "w"):
        pass

[f for f in os.listdir() if f.startswith("moj_test_foo_")]

['my_test_foo_1',
 'my_test_foo_6',
 'my_test_foo_8',
 'my_test_foo_9',
 'my_test_foo_7',
 'my_test_foo_0',
 'my_test_foo_5',
 'my_test_foo_2',
 'my_test_foo_3',
 'my_test_foo_4']

In [26]:
for i in range(10):
    with open("moj_test_bar_{}".format(i), "w"):
        pass

[f for f in os.listdir() if f.startswith("moj_test_bar_")]

['my_test_bar_4',
 'my_test_bar_3',
 'my_test_bar_2',
 'my_test_bar_5',
 'my_test_bar_0',
 'my_test_bar_7',
 'my_test_bar_9',
 'my_test_bar_8',
 'my_test_bar_6',
 'my_test_bar_1']

Przed użyciem nazw plików warto je posortować:

In [27]:
filenames = os.listdir()
filenames.sort()

In [28]:
[f for f in filenames if f.startswith("moj_test_foo_")]

['my_test_foo_0',
 'my_test_foo_1',
 'my_test_foo_2',
 'my_test_foo_3',
 'my_test_foo_4',
 'my_test_foo_5',
 'my_test_foo_6',
 'my_test_foo_7',
 'my_test_foo_8',
 'my_test_foo_9']

In [29]:
for f in os.listdir():
    if f.startswith("moj_test_foo_") or f.startswith("moj_test_bar_"):
        os.remove(f)

Mam nadzieję, że spodobał Ci się ten notatnik. Jeżeli nie uzyskujesz odtwarzalnych rezultatów lub jeśli są one inne od moich, to [zgłoś problem](https://github.com/ageron/handson-ml2/issues) w serwisie GitHub: podaj wersję Pythona, modułów TensorFlow i NumPy, a także używany system operacyjny. Dziękuję!

Jeżeli chcesz dowiedzieć się więcej na temat uczenia głębokiego i modułu TensorFlow, sprawdź moją książkę [Uczenie maszynowe z użyciem Scitkit-Learn i TensorFlow](https://helion.pl/ksiazki/uczenie-maszynowe-z-uzyciem-scikit-learn-i-tensorflow-aur-lien-g-ron,uczema.htm#format/d), O'Reilly. Możesz także obserwować mnie na Twitterze: [@aureliengeron](https://twitter.com/aureliengeron) i oglądać moje prezentacje [w serwisie YouTube](https://www.youtube.com/c/AurelienGeron).